In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Configuration

BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LR_RATE = 3e-4
MOMENTUM = 0.9
NUM_EPOCHS = 5

In [ ]:
# ---> Loading the Dataset

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)



100%|██████████| 170498071/170498071 [00:03<00:00, 48995108.11it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class CIFAR10_CNN(nn.Module):
  def __init__(self):
    super(CIFAR10_CNN, self).__init__()
    # Convolutional layers
    self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=5, padding=2)

    # Fully connected layers
    self.fc1 = nn.Linear(128 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10) # 10 classes for CIFAR-10

  def forward(self, x):
    # Forward pass
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1, 128 * 4 * 4) # Flatten the data for the fully connected layer
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return F.log_softmax(x, dim=1)

# Create am instance of the CNN

model = CIFAR10_CNN().to(DEVICE)

In [ ]:
# Loss funcion
loss_function = nn.CrossEntropyLoss()

# Optimizer (SGD or Adam)
sgd = optim.SGD(model.parameters(), lr=LR_RATE, momentum=MOMENTUM)

adam = optim.Adam(model.parameters(), lr=LR_RATE)

In [ ]:
# Training

for e in range(NUM_EPOCHS):
  model.train()
  loop = tqdm(enumerate(train_loader))
  for i, (data, target) in loop:
    # Forward pass
    data, target = data.to(DEVICE), target.to(DEVICE)
    output = model(data)

    # Backwardprop
    loss = loss_function(output, target)
    adam.zero_grad()
    loss.backward()
    adam.step()
    loop.set_postfix(loss = loss.item())

782it [00:22, 35.48it/s, loss=1.31]
782it [00:21, 37.13it/s, loss=0.727]
782it [00:21, 36.29it/s, loss=0.579]
782it [00:20, 37.72it/s, loss=0.744]
782it [00:21, 36.93it/s, loss=0.874]


In [ ]:
# Testing loop

correct = 0
model.eval()
with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(DEVICE), target.to(DEVICE)
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).sum()

print(f'Accuracy: {100. * correct / len(test_loader.dataset)}%')

Accuracy: 70.0999984741211%
